In [34]:
from datasets import load_dataset
import json
# dataset = load_dataset("ZHUI/alpaca_demo", split="train")

In [46]:
with open("stage1/train1.json","r",encoding="utf8") as f:
    train_dataset = json.load(f)
with open("stage1/dev1.json","r",encoding="utf8") as f:
    dev_dataset = json.load(f)
with open("stage1/test1.json","r",encoding="utf8") as f:
    test_dataset = json.load(f)

In [41]:
train_dataset[0]

{'source': 'FabNER',
 'sentence': '“截至9月末，深圳现金累计投放量同比出现负数。”近日，一位接近监管部门人士对本报记者称，“',
 'coarse_types': ['生物', '职位', '科学', '组织机构', '学历', '位置'],
 'entities': [{'name': '记者', 'coarse_type': '职位', 'fine_type': '概念'}]}

In [47]:
train_sum,dev_sum=0,0
for i,unit in enumerate(train_dataset):
    for k in unit["entities"]:
        if k["coarse_type"] not in unit["coarse_types"]:
            print(i,unit)
            train_sum+=1
            unit["coarse_types"].append(k["coarse_type"])
for i,unit in enumerate(dev_dataset):
    for k in unit["entities"]:
        if k["coarse_type"] not in unit["coarse_types"]:
            print(i,unit)
            dev_sum+=1
            unit["coarse_types"].append(k["coarse_type"])

15 {'source': 'FabNER', 'sentence': 'It is unclear if the move was due to a commercial decision , or a result of usual censorship by the State Administration of Press , Publication , Radio , Film and Television .', 'coarse_types': ['food', 'organization', 'science', 'location'], 'entities': [{'name': 'State Administration of Press , Publication , Radio , Film and Television', 'coarse_type': 'organization', 'fine_type': 'government'}, {'name': 'State Administration of Press , Publication , Radio , Film and Television', 'coarse_type': 'politics', 'fine_type': 'government'}]}
19 {'source': 'FabNER', 'sentence': '1963年11月出生，汉族，中共党员，大学专科学历，会计师。', 'coarse_types': ['职位', '组织机构', '学历', '文学', '位置'], 'entities': [{'name': '汉族', 'coarse_type': '组织机构', 'fine_type': '民族'}, {'name': '中共党员', 'coarse_type': '职位', 'fine_type': '概念'}, {'name': '会计师', 'coarse_type': '职位', 'fine_type': '概念'}, {'name': '大学专科学历', 'coarse_type': '科学', 'fine_type': '学历'}]}
27 {'source': 'FabNER', 'sentence': '战斗机数量占海军和海军陆战队a型

In [48]:
train_sum,dev_sum

(2990, 57)

In [50]:
with open("stage1/train2.json","w",encoding="utf8") as file:
    file.write(json.dumps(train_dataset,ensure_ascii=False))
with open("stage1/dev2.json","w",encoding="utf8") as file:
    file.write(json.dumps(dev_dataset,ensure_ascii=False))

In [70]:
entities = []
for unit in train_dataset:
    for t in unit["entities"]:
        entities.append(t)

In [72]:
import pandas as pd
entities = pd.DataFrame(entities)

In [82]:
entities.groupby(['coarse_type',"fine_type"]).count().to_csv("实体粗细粒度映射关系.csv")

In [78]:
entities

name coarse_type fine_type
0                     记者          职位        概念
1      C4 - C - N - PEG9     science  chemical
2                 C12EO4     science  chemical
3          triglycerides     science  chemical
4              metformin     science  chemical
...                  ...         ...       ...
41054        glutathione     science  chemical
41055               泰禾红御        组织机构        公司
41056                阿拉伯          位置        国家
41057          《没有青春的青春》          产品        电影
41058              youth          产品        电影

[41059 rows x 3 columns]

In [29]:
train_dataset[-1],dev_dataset[-1],test_dataset[-1]

({'source': 'FabNER',
  'sentence': 'The effect of powder chemical composition on the microstructural evolution of SLM processed 17-4 PH in the aB-built condition was studied .',
  'coarse_types': ['product', 'science'],
  'entities': []},
 {'source': 'FabNER',
  'sentence': 'Across substrates , substrate orthologs and conserved sequence stretches can also be shown , with iceLogo visualization provided for the latter .',
  'coarse_types': ['science'],
  'entities': []},
 {'id': 'a64272f8-ebe0-53c4-819e-923d727c9d25',
  'domain': 'science',
  'sentence': '梁同实验室的当前研究集中在涉及脂肪酸代谢的酶上，包括乙酰辅酶A羧化酶、肉碱酰基转移酶、AMP激活的蛋白激酶等。',
  'coarse_types': ['文学', '人', '产品', '事件', '科学']})

In [52]:
import json

def format_entities(entities):
    """格式化实体输出"""
    if not entities:
        return "未识别到实体"
    
    entity_lines = []
    for i, entity in enumerate(entities, 1):
        entity_lines.append(f"{i}. 实体：{entity['name']}")
        entity_lines.append(f"   粗类型：{entity['coarse_type']}")
        entity_lines.append(f"   细类型：{entity['fine_type']}")
    
    return "\n".join(entity_lines)

def prepare_finetuning_data(input_file, output_file):
    """准备微调数据"""
    
    # 读取原始数据
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    processed_data = []
    
    for item in data:
        # 构建输入
        sentence = item['sentence']
        coarse_types = "、".join(item['coarse_types'])
        
        input_text = f"文本：{sentence}\n\n可选的实体类型：{coarse_types}"
        
        # 构建输出
        entities_output = format_entities(item['entities'])
        output_text = f"实体识别结果：\n{entities_output}"
        
        # 构建训练样本
        training_sample = {
            "instruction": "请从给定文本中识别出所有命名实体，并按照指定的实体类型进行分类。",
            "input": input_text,
            "output": output_text
        }
        
        processed_data.append(training_sample)
    
    # 保存处理后的数据
    with open(output_file, 'w', encoding='utf-8') as f:
        for item in processed_data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    print(f"处理完成！共生成 {len(processed_data)} 条训练数据")
    return processed_data

input_file = "stage1/train2.json"
output_file = "ner_finetuning_train.jsonl"
processed_data1 = prepare_finetuning_data(input_file, output_file)
input_file = "stage1/dev2.json"
output_file = "ner_finetuning_dev.jsonl"
processed_data2 = prepare_finetuning_data(input_file, output_file)

处理完成！共生成 26500 条训练数据
处理完成！共生成 545 条训练数据


In [64]:
processed_data2[0]

{'instruction': '请从给定文本中识别出所有命名实体，并按照指定的实体类型进行分类。',
 'input': '文本：After renewing the cylinder assembly , it was tested around Brighton and Eastleigh using an LNER Dynamometer car , where good running was experienced at high costs in fuel and effort on the part of the fireman .\n\n可选的实体类型：organization、science、politics、location、event',
 'output': '实体识别结果：\n1. 实体：Brighton\n   粗类型：location\n   细类型：city\n2. 实体：Eastleigh\n   粗类型：location\n   细类型：town'}

In [62]:
print(processed_data2[3]["output"])


实体识别结果：
未识别到实体


In [ ]:
# 粗粒度对应的细粒度有哪些？


In [85]:

# 读取原始数据
with open("stage1/test1.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

processed_data = []

for item in data:
    # 构建输入
    sentence = item['sentence']
    coarse_types = "、".join(item['coarse_types'])
    
    input_text = f"文本：{sentence}\n\n可选的实体类型：{coarse_types}"
    
    # 构建预测样本
    training_sample = {
        "instruction": "请从给定文本中识别出所有命名实体，并按照指定的实体类型进行分类。",
        "input": input_text,
        "output":""
    }
        
    processed_data.append(training_sample)

# 保存处理后的数据
with open("ner_finetuning_predict.jsonl", 'w', encoding='utf-8') as f:
    for item in processed_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"处理完成！共生成 {len(processed_data)} 条训练数据")

处理完成！共生成 4962 条训练数据


In [65]:
test_dataset[0]

{'id': 'c9946072-c4c5-5eb4-9eed-a1aa0373efe1',
 'domain': 'ai',
 'sentence': 'Troponymy is one of the possible relations between verbs in the semantic network of the WordNet database.',
 'coarse_types': ['person',
  'politics',
  'location',
  'literature',
  'product',
  'computer science']}

In [ ]:
# 训练：
import paddle
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import AutoTokenizer, AutoModelForCausalLM
from paddlenlp.trainer import Trainer, TrainingArguments

# 加载tokenizer和模型
model_name = "qwen/qwen2.5-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def preprocess_function(examples):
    """数据预处理函数"""
    
    # 构建输入
    inputs = []
    for instruction, input_text in zip(examples['instruction'], examples['input']):
        prompt = f"<|im_start|>system\n{instruction}<|im_end|>\n<|im_start|>user\n{input_text}<|im_end|>\n<|im_start|>assistant\n"
        inputs.append(prompt)
    
    # Tokenize
    model_inputs = tokenizer(
        inputs,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pd"
    )
    
    # Tokenize labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['output'],
            max_length=512,
            truncation=True,
            padding=True,
            return_tensors="pd"
        )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 加载数据集
def read_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            yield json.loads(line)

train_dataset = load_dataset(read_data, data_files="stage1/ner_finetuning_train.jsonl")
dev_dataset = load_dataset(read_data, data_files="stage1/ner_finetuning_dev.jsonl.jsonl")

# 预处理数据集
train_dataset = train_dataset.map(preprocess_function, batched=True)
dev_dataset = dev_dataset.map(preprocess_function, batched=True)

# 训练参数
training_args = TrainingArguments(
    output_dir="./qwen2.5-7b-ner",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    warmup_steps=100,
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    remove_unused_columns=False,
    fp16=True,
)

# 创建Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    dev_dataset=dev_dataset,
    tokenizer=tokenizer,
)

# 开始训练
trainer.train()
trainer.save_model()

In [ ]:
def predict_entities(text, coarse_types):
    """使用微调后的模型进行实体识别"""
    
    instruction = "请从给定文本中识别出所有命名实体，并按照指定的实体类型进行分类。"
    input_text = f"文本：{text}\n\n可选的实体类型：{coarse_types}"
    
    prompt = f"<|im_start|>system\n{instruction}<|im_end|>\n<|im_start|>user\n{input_text}<|im_end|>\n<|im_start|>assistant\n"
    
    inputs = tokenizer(prompt, return_tensors="pd")
    
    with paddle.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=1024,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 提取assistant的回复
    assistant_response = response.split("<|im_start|>assistant\n")[-1]
    
    return assistant_response

# 使用示例
text = "After renewing the cylinder assembly , it was tested around Brighton and Eastleigh using an LNER Dynamometer car"
coarse_types = ["organization", "science", "politics", "location", "event"]

result = predict_entities(text, coarse_types)
print(result)

In [84]:
import re
import json

def convert_entities_to_json(text):
    """
    将实体识别文本转换为JSON格式
    """
    # 正则表达式匹配实体信息
    pattern = r'(\d+)\.\s*实体：(.+?)\s*粗类型：(.+?)\s*细类型：(.+?)(?=\n\d+\.|\n*$)'
    matches = re.findall(pattern, text, re.DOTALL)
    
    result = []
    for match in matches:
        # 序号, 实体名, 粗类型, 细类型
        _, name, coarse_type, fine_type = match
        
        # 清理可能的空白字符
        name = name.strip()
        coarse_type = coarse_type.strip()
        fine_type = fine_type.strip()
        
        # 添加到结果列表
        result.append({
            "name": name,
            "coarse_type": coarse_type,
            "fine_type": fine_type
        })
    
    return result

# 测试数据
input_text = """实体识别结果：\n1. 实体：卡迪尔加马尔\n   粗类型：人\n   细类型：政治家\n2. 实体：卡迪尔加马尔\n   粗类型：政治\n   细类型：政治家\n3. 实体：斯里兰卡\n   粗类型：位置\n   细类型：国家\n4. 实体：南亚\n   粗类型：位置\n   细类型：洲"""

# 转换并输出
output = convert_entities_to_json(input_text)
print(json.dumps(output, ensure_ascii=False, indent=2))

[
  {
    "name": "卡迪尔加马尔",
    "coarse_type": "人",
    "fine_type": "政治家"
  },
  {
    "name": "卡迪尔加马尔",
    "coarse_type": "政治",
    "fine_type": "政治家"
  },
  {
    "name": "斯里兰卡",
    "coarse_type": "位置",
    "fine_type": "国家"
  },
  {
    "name": "南亚",
    "coarse_type": "位置",
    "fine_type": "洲"
  }
]
